In [26]:
import numpy as np
import string
import random
from Levenshtein import distance
from scipy.spatial.distance import pdist, squareform

### Generate random strings

In [3]:
# "B", "Z", 
amino_acids = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
amino_acids_np = np.array(amino_acids)
# len(amino_acids) # = 20

n_nodes = 50
str_len = 10

In [9]:
def charr_to_string(charr):
    return "".join(charr)

# charr_to_string(["A", "B", "C"])

# to implement
def string_to_charr(str):
    return

'ABC'

In [4]:
# --- purely random strings; high hamming distances --- #
random_strings = np.random.choice(amino_acids_np, (n_nodes, str_len))
# print(random_strings)

In [24]:
# --- get smaller hamming distances; sub-alphabet --- #
# array of character arrays
amino_acids_subset = amino_acids_np[0:2] # uses only first 2 letters
random_charr_subset = np.random.choice(amino_acids_subset, (n_nodes, str_len))

# array of strings
random_strings_subset = np.array([])
for charr in random_charr_subset:
    random_strings_subset = np.append(random_strings_subset, charr_to_string(charr))

random_strings_subset = random_strings_subset.reshape(-1,1)

print(random_strings_subset.shape)

(50, 1)


### Hamming distance on char arrays

In [35]:
# --- Method 1: trick on charr; hamming only --- #
# https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix
hamming_1 = (random_charr_subset[:, None, :] != random_charr_subset).sum(2)
print(hamming_1)

[[0 3 4 ... 7 6 6]
 [3 0 7 ... 6 3 5]
 [4 7 0 ... 3 8 4]
 ...
 [7 6 3 ... 0 5 3]
 [6 3 8 ... 5 0 4]
 [6 5 4 ... 3 4 0]]


In [44]:
# --- Method 2: scipy distance --- #
test = pdist(random_strings_subset, lambda x,y: hamming_distance_str(x[0], y[0])) # grab str1 from [str1]
print(squareform(test))

# can use a custom distance function, just need to wrap it in a lambda(?)



[[0. 3. 4. ... 7. 6. 6.]
 [3. 0. 7. ... 6. 3. 5.]
 [4. 7. 0. ... 3. 8. 4.]
 ...
 [7. 6. 3. ... 0. 5. 3.]
 [6. 3. 8. ... 5. 0. 4.]
 [6. 5. 4. ... 3. 4. 0.]]


In [38]:
# --- Method 3: manual implementation (charr) --- #

def hamming_distance_charr(arr1, arr2):
    assert len(arr1) == len(arr2)

    hamming_dist = 0

    # customizable
    # can consider representing characters as one-hot encodings if useful. matrix multiplication? 
    for i in range(len(arr1)):
        if arr1[i] != arr2[i]:
            hamming_dist += 1

    return hamming_dist


hamming_matrix = np.ndarray((n_nodes, n_nodes))

for i in range(n_nodes):
    for j in range(n_nodes):
        hamming_matrix[i][j] = hamming_distance_charr(random_charr_subset[i], random_charr_subset[j])

# print(hamming_matrix)

In [41]:
# --- Method 4: manual implementation (charr) --- #
# https://stackoverflow.com/questions/54172831/hamming-distance-between-two-strings-in-python

# basic version
def hamming_distance_str(str1, str2):
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))

# hamming_distance_str("hello", "hallo")

# customizable weights
def hamming_distance_str_custom(str1, str2):
    assert len(str1) == len(str2)

    hamming_dist = 0
    for i in range(len(str1)):
        if str1[i] != str2[i]:
            hamming_dist += 1 # customizable; matrix even

    return hamming_dist

### Levenshtein Distance

In [38]:
# --- Method 1: recursion --- #

def compute_LD(arr1, arr2):
    # costs; customizable
    sub_cost = 1
    ins_cost = 1

    # base case
    if len(arr1) == 0:
        return ins_cost * len(arr2)
    
    if len(arr2) == 0:
        return ins_cost * len(arr1)
    
    # 3 cases: insert first element of arr1, insert first of arr2, or substitute two firsts
    arr1_insert_cost = compute_LD(arr1[1:], arr2) + ins_cost
    arr2_insert_cost = compute_LD(arr1, arr2[1:]) + ins_cost
    sub_first_cost = compute_LD(arr1[1:], arr2[1:])
    if arr1[0] != arr2[0]:
        sub_first_cost += sub_cost

    return min(arr1_insert_cost, arr2_insert_cost, sub_first_cost)

# A B - D - T
# A B C D G T
a = ["A", "B", "D", "T"]
b = ["A", "B", "C", "D", "G", "T"]
compute_LD(a, b)


2

In [33]:
# --- Method 2: Levenshtein package --- #
a = ["A", "B", "D", "T"]
b = ["A", "B", "C", "D", "G", "T"]
print(distance(a,b)) # Levenshtein.distance

2
